In [ ]:
# Imports und Grundsetup
import os
import sys
from pathlib import Path

sys.path.append('.')

# Modulare Komponenten importieren
#from config.pipeline_configs import get_baseline_config
#from core.rag_pipeline import RAGPipeline

# API-Schlüssel prüfen
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  OPENAI_API_KEY nicht gesetzt!")
    print("Setzen Sie Ihren API-Schlüssel:")
    print("os.environ['OPENAI_API_KEY'] = 'sk-your-key-here'")
    # Uncomment und Key einfügen:
    # os.environ['OPENAI_API_KEY'] = 'sk-your-key-here'
else:
    print("✅ OpenAI API Key gefunden:")
    #Show part of the key (first 10 and last 4 characters)
    print(os.getenv("OPENAI_API_KEY")[:10] + "..." + os.getenv("OPENAI_API_KEY")[-4:])

print("✅ Setup abgeschlossen")


✅ OpenAI API Key gefunden
sk-proj-72...SDwA
✅ Setup abgeschlossen


In [ ]:
# Baseline-Konfiguration laden
config = get_baseline_config()
print(f"📋 Konfiguration: {config.get_component_types()}")

# Echte RAG-Pipeline erstellen
pipeline = RAGPipeline(config)
print("✅ Modulare RAG-Pipeline erstellt")


In [ ]:
# DSGVO-Dokument laden (relativ zum Projektroot)
dsgvo_file = "../data/raw/dsgvo.txt"

# Prüfen ob Datei existiert
if not os.path.exists(dsgvo_file):
    print(f"❌ Datei nicht gefunden: {dsgvo_file}")
    print("Erstelle Beispiel-Datei...")
    
    # Verzeichnis erstellen
    Path(dsgvo_file).parent.mkdir(parents=True, exist_ok=True)
    
    # Beispiel-DSGVO-Text
    sample_text = """
VERORDNUNG (EU) 2016/679 - DATENSCHUTZ-GRUNDVERORDNUNG

Artikel 1 - Gegenstand und Ziele
(1) Diese Verordnung enthält Vorschriften zum Schutz natürlicher Personen bei der Verarbeitung personenbezogener Daten.
(2) Diese Verordnung schützt die Grundrechte und Grundfreiheiten natürlicher Personen.

Artikel 83 - Geldbußen
(5) Verstöße gegen bestimmte Bestimmungen werden mit Geldbußen von bis zu 20 000 000 EUR oder 4 % des weltweiten Jahresumsatzes geahndet.

Artikel 7 - Bedingungen für die Einwilligung
(1) Beruht die Verarbeitung auf einer Einwilligung, muss der Verantwortliche nachweisen können, dass die betroffene Person in die Verarbeitung eingewilligt hat.

Artikel 15 - Auskunftsrecht der betroffenen Person
(1) Die betroffene Person hat das Recht, eine Bestätigung darüber zu verlangen, ob sie betreffende personenbezogene Daten verarbeitet werden.
    """
    
    with open(dsgvo_file, 'w', encoding='utf-8') as f:
        f.write(sample_text)
    
    print("✅ Beispiel-Datei erstellt")

# Dokument laden und indexieren
documents = pipeline.load_documents_from_file(dsgvo_file)
print(f"📄 {len(documents)} Dokument(e) geladen")

# Indexierung
print("🔄 Indexiere Dokumente...")
stats = pipeline.index_documents(documents)
print(f"✅ {stats['total_chunks']} Chunks indexiert")


In [ ]:
# Einfache Abfrage-Funktion
def frage_stellen(frage):
    """Stellt eine Frage an das RAG-System."""
    print(f"❓ Frage: {frage}")
    print("-" * 50)
    
    result = pipeline.query(frage)
    
    print(f"💬 Antwort: {result['answer']}")
    print(f"⏱️  Zeit: {result['query_time']:.2f}s")
    print("=" * 50)
    
    return result

# Beispiel-Frage
frage_stellen("Was ist die maximale Geldbuße nach der DSGVO?")


In [ ]:
# Weitere Beispiel-Fragen
beispiel_fragen = [
    "Welche Rechte haben betroffene Personen?",
    "Was ist eine Einwilligung?",
    "Was regelt Artikel 1 der DSGVO?"
]

for frage in beispiel_fragen:
    frage_stellen(frage)
    print()


In [ ]:
# Eigene Frage stellen
# Ändern Sie hier Ihre Frage:
meine_frage = "Ihre Frage hier eingeben"

# Frage stellen (nur ausführen wenn Frage geändert wurde)
if meine_frage != "Ihre Frage hier eingeben":
    frage_stellen(meine_frage)
else:
    print("💡 Ändern Sie 'meine_frage' oben und führen Sie die Zelle erneut aus")


In [ ]:
# Pipeline-Status anzeigen
print("📊 Pipeline-Status:")
print(f"  Indexiert: {'✅' if pipeline.is_indexed else '❌'}")
print(f"  Dokumente: {pipeline.indexed_document_count}")

# Komponenten anzeigen
print("\n🔧 Komponenten:")
components = pipeline.get_component_info()
for name, info in components.items():
    print(f"  {name}: {info['type']}")

print("\n✅ System bereit für Experimente!")
